# Tutorial de Big Data (UdeSA) 2025
## Tutorial 6

**Objetivo:** Correr regresiones lineales. Estimar polinomios y encontrar el ECM.

Veremos:
- Repaso de Numpy y Scikit-learn
- Regresión lineal y polinómica
- Estadísticas (similares a stata o R)
- Encontrar el ECM

##  Repaso: NumPy y scikit-learn                   
**El paquete NumPy** es fundamental en Python. Está escrito en lenguajes de bajo nivel, lo que permite realizar operaciones matemáticas de manera muy eficiente. Para más información, ver la [guía oficial de uso de NumPy](https://docs.scipy.org/doc/numpy/user/index.html).

**El paquete scikit-learn** es una biblioteca de Python usada para machine learning, construida encima de NumPy y otros paquetes. Permite procesar datos, reducir la dimensionalidad de la base, implementar regresiones, clasificaciones, clustering y más. Pueden ver la [web de scikit-learn](https://scikit-learn.org/stable/)


In [ ]:
# Importamos paquetes
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# Correr si les tira que necesitan la ultima version de pandas. 

#En mi caso, necesitaba el update de pandas
#import sys
#!{sys.executable} -m pip install --upgrade pandas

### REGRESIÓN LINEAL CON SCIKIT-LEARN
Ahora utilizaremos la función [LinearRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) del paquete scikit-learn.
    
Se pueden proveer muchos parámetros opcionales para esta función:

- **fit_intercept**: Booleano que decide si calcular el intercepto (True) o considerarlo igual a cero (False). Por default es True.
- **normalize**: Booleano que decide si normalizar las variables input (True) o no (False). Es False por default.
- **copy_X**: Booleano que decide si copiar (True) o sobreescribir las variables input (False). Es True por default.

In [ ]:
# Creamos unos datos de ejemplo
x = np.array([5, 15, 25, 35, 45, 55])
y = np.array([5, 20, 14, 32, 22, 38])

print(x)
print(y)
# Ambos son vectores fila

In [ ]:
# Repasamos el Reshape para transformar x en un vector columna
x = x.reshape((-1, 1))   # El -1 indica el largo del array
# Es equivalente a: x = x.reshape((6, 1))

print(x)
print(y)

In [ ]:
# Primero, estimar el modelo. Lo hacemos con fit():
model = LinearRegression().fit(x, y)

In [ ]:
# Veamos ahora los resultados

# Calculamos el R2
r2 = model.score(x, y)
print('Coeficiente de determinación:', r2)

# El intercepto
intercepto = model.intercept_
print('\nIntercepto:', intercepto)

# La pendiente
pendiente = model.coef_
print('\nPendiente:', pendiente)


In [ ]:
# Hacemos un scatter plot
plt.plot(x, y, 'o')
plt.plot(x, pendiente*x + intercepto)


In [ ]:
# Supongamos que ahora queremos predecir con este modelo.
# Aplicamos .predict(), metemos los valores del regresor en el modelo estimado y
# obtenemos la correspondiente respuesta predicha.

y_pred = model.predict(x)
print('Respuesta predicha:\n', y_pred)

# Recordemos cómo era nuestro vector y
print('\nEl vector de y:', y)

# Con la y predicha podemos calcular el R^2 de esta otra forma
r2_new = r2_score(y, y_pred)
print("\nResultado anterior:", r2, "\nResultado nuevo:", r2_new)

In [ ]:
# Si quiero probar valores nuevos de x (no los que usé para estimar el modelo):
x_new = np.arange(start=10, stop=20, step=2).reshape((-1, 1))   # Generamos valores entre [10, 20), con saltos de 2 en 2
print(x_new)

y_pred_new = model.predict(x_new)
print('\nNueva respuesta predicha:\n', y_pred_new)


In [ ]:
# Para regresión lineal múltiple es lo mismo:
# Armamos un vector para la variable dependiente y una matriz de regresores:
x = np.array([[0, 1], [5, 1], [15, 2], [25, 5], [35, 11], [45, 15], [55, 34], [60, 35]])

y =  np.array([4, 5, 20, 14, 32, 22, 38, 43])

print(x)
print(y)


In [ ]:
# Estimamos el modelo
model = LinearRegression().fit(x, y)
r2 = model.score(x, y)

# Miramos resultados
print('Coeficiente de determinación:', r2)
print('\nIntercepto:', model.intercept_)
print('\nCoeficientes:', model.coef_)

In [ ]:
# Vemos la respuesta predicha para los valores originales de los regresores
y_pred = model.predict(x)
print('Respuesta predicha:', y_pred, sep='\n')

In [ ]:
# Vemos la predicción para nuevos valores de X
x_new = np.arange(start=1, stop=31, step=3).reshape((-1, 2))   # Matriz con 2 columnas y tantas filas como tenga el array
print(x_new)
y_new = model.predict(x_new)
print('Nueva respuesta predicha:', y_new, sep='\n')

### REGRESIÓN POLINÓMICA
Si queremos correr una regresión de y contra x y x^2 necesitamos generar los datos de la nueva variable independiente.

[PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) (de scikitlearn) genera una nueva matriz que consta de todas las combinaciones polinomiales de las x con un grado menor o igual al grado especificado. Por ejemplo, si una muestra de entrada es bidimensional y de la forma [a, b], las características polinomiales de grado 2 son [1, a, b, a^2, ab, b^2].

Se pueden cambiar varios parámetros de PolynomialFeatures:

- **degree** es un entero (2 por default) que representa el grado de la función de regresión polinómica.

- **include_bias** es un Booleano (True por default) que decide si incluir la columna de 1 que corresponde al intercepto (True) o no (False).


In [ ]:
x = np.array([5, 15, 25, 35, 45, 55]).reshape((-1, 1))   # ¿Por qué reshape?
y = np.array([15, 11, 2, 8, 25, 32])   # Vector fila
print(x)
print(y)

In [ ]:
model_pol = PolynomialFeatures(include_bias=False)
model_pol.fit(x)
x_ = model_pol.transform(x)   # Aplicamos transformación a los datos. Genera una matriz de 6x2 (con x y x^2)
# equivalente a: x_ = model_pol.fit_transform(x)
print(x_)

In [ ]:
# Ahora usamos x_ para correr la regresión
model_pol = LinearRegression().fit(x_, y)

In [ ]:
print('Coeficiente de determinación:', model_pol.score(x_, y))
print('\nIntercepto:', model_pol.intercept_)
print('\nCoeficientes:', model_pol.coef_)

### Imitando a Stata con statsmodels

[statsmodels](https://www.statsmodels.org/stable/index.html) proporciona clases y funciones para la estimación de modelos estadísticos, para realizar pruebas estadísticas y para explorar datos estadísticos.

In [ ]:
x = [[0, 1], [5, 1], [15, 2], [25, 5], [35, 11], [45, 15], [55, 34], [60, 35]]
y = [4, 5, 20, 14, 32, 22, 38, 43]
x, y = np.array(x), np.array(y)

x = sm.add_constant(x)
print(x)
print(y)

In [ ]:
# Especificamos el modelo
model = sm.OLS(y, x)
# Ajustamos el modelo
results = model.fit()

print(results.summary())

In [ ]:
# Si solo queremos ver los coeficientes
print(results.params)

In [ ]:
# También lo podemos imprimir los resultados para latex
print(results.summary().as_latex())

In [ ]:
# También lo podemos imprimir los resultados como CSV
print(results.summary().as_csv())

In [ ]:
# Se puede obtener la respuesta predicha con los valores de x utilizados en el
# entrenamiento del modelo usando .fittedvalues o .predict():

print('predicted response:\n', results.fittedvalues) #equivalente: results.predict(x)

###     ERROR CUADRÁTICO MEDIO

Ahora veamos algunas métricas de evaluación usuales para los problemas de regresión en Machine Learning.

Vamos a observar los valores de las siguientes métricas:

**Error Cuadrático Medio / Mean Squared Error**

$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{f}(x_i))^2$

**Raíz del Error Cuadrático Medio / Root Mean Squared Error**

$RMSE = \sqrt{MSE}$

**Error Absoluto Medio / Mean Absolute Error**

$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{f}(x_i)|$


In [ ]:
# Generamos un dataset aleatorio
np.random.seed(0)
x = np.random.rand(100, 1)  # Array de la forma (100, 1) con nros aleatorios entre [0, 1) de una distribución uniforme
y = 2 + 3*x + np.random.rand(100, 1)
print("x:\n", x)
print("y:\n", y)

In [ ]:
# Graficamos
plt.scatter(x, y, s=10)  # s indica el tamaño de los puntos del scatter.
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
x = sm.add_constant(x)
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

In [ ]:
# Predecimos las y
y_pred = results.predict(x)
y_pred

In [ ]:
# Vemos el MSE

# Usando MSE de scikit-learn
mse1 = mean_squared_error(y, y_pred)
print(mse1)

# Usando Numpy
mse2 = np.square(np.subtract(y, y_pred)).mean()
print(mse2)
# Cuidado! Nos dio distinto porque las funciones de Numpy no trabajan igual con arrays.
# Veamos cómo solucionar esto

In [ ]:
y_flat = y.flatten()   # Modificamos la forma de y
print(y.shape, y_flat.shape)
print("y flat:", y_flat)
print("y:",  y)

In [ ]:
# Volvamos a probar
# Usando MSE de scikit-learn
mse1 = mean_squared_error(y_flat, y_pred)
print(mse1)

# Usando Numpy
mse2 = np.square(np.subtract(y_flat, y_pred)).mean()
print(mse2)
# Ahora sí obtenemos el mismo resultado

In [ ]:
# También podemos ver el RMSE y el MAE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(rmse)
mae = mean_absolute_error(y, y_pred)
print(mae)